In [6]:
#coding=utf-8
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF 
from sklearn import svm
train=pd.read_csv('train.csv',sep='\t')
test=pd.read_csv('test_new.csv')

In [7]:
import re
def extractChinese(s):
    pattern="[\u4e00-\u9fa5]+"#中文正则表达式
    regex = re.compile(pattern) #生成正则对象 
    results = regex.findall(s) #匹配
    return "". join(results)
# 预处理数据
label = train['label']
train_data = []
for i in range(len(train['comment'])):
    train_data.append(' '.join(extractChinese(train['comment'][i])))
test_data = []
for i in range(len(test['comment'])):
    test_data.append(' '.join(extractChinese(test['comment'][i])))

In [10]:
tfidf = TFIDF(min_df=1, # 最小支持长度
           max_features=150000,#取特征数量
           strip_accents ='unicode',
           analyzer='word',
           token_pattern=r'\w{1,}',
           ngram_range=(1, 3),  
           use_idf=1,
           smooth_idf=1,
           sublinear_tf=1,
           stop_words=None,
  
           ) 

In [11]:
data_all = train_data + test_data
len_train = len(train_data)

tfidf.fit(data_all)

data_all = tfidf.transform(data_all)

# 恢复成训练集和测试集部分
train_x = data_all[:len_train]
test_x = data_all[len_train:]
print ('TF-IDF处理结束.')

TF-IDF处理结束.


In [12]:
clf=svm.LinearSVC(loss='squared_hinge', dual=True, tol=0.0001,
                  C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1, 
                  class_weight='balanced', verbose=0, random_state=None, max_iter=1000)
svm=clf.fit(train_x,label)
svm_pre=svm.predict(test_x)
svm = pd.DataFrame(data=svm_pre, columns=['comment'])
svm['id'] = test.id
svm = svm[['id', 'comment']]
svm.to_csv('svm.csv',index=False)

In [13]:
r=test
r['id']=svm_pre
r0=r[r.id==0]

In [14]:
key_word=['蚊子','老鼠','苍蝇','酸臭']

# key_word2=['蚊子','剩','不新鲜','没熟','老鼠','烂','骚味','苍蝇','虫','臭','想吐','太硬']
for i in key_word:
  print (r0[r0['comment'].str.contains(i)])

      id                                        comment
17     0  天，还有蚊子，份量太太太太少了，老大你这个好几十哦，全是西海椒和芹菜，老板这样做生意不好哦
1753   0  去了两次，人太多，团购的上菜较慢，有苍蝇兄也有蚊子兄，总的来说算不错了！喜欢喝老鸭汤主要！
     id                    comment
722   0  想吐，银耳汤杯盖上面有一颗老鼠屎，而且还黏在上面了
      id                                        comment
1342   0             有苍蝇。。。。。。对于一个才学了节肢动物的人来说  真的是人都不好了
1753   0  去了两次，人太多，团购的上菜较慢，有苍蝇兄也有蚊子兄，总的来说算不错了！喜欢喝老鸭汤主要！
      id                                            comment
531    0  第一次没看评价点了这家，因为图片和价位还蛮吸引人的，吃完没事干给你们来个具体的建议，是真的不...
1296   0  这家冒菜确实难吃。 盐巴像打倒了一样 把我咸安逸了。这都算了嘛 关键是菜不晓得放了好多天的了...


In [15]:
key_word2=['剩','不新鲜','没熟','烂']
for i in key_word2:
  print (r0[r0['comment'].str.contains(i)])

      id                                            comment
74     0  除了尖椒鸡外，其他图片与实物完全不符，全是干辣椒，而且土豆片中有葱有蒜台，感觉像是几个菜混合...
229    0                               饭像没熟样 好像昨天的剩饭 一点都不好吃
299    0                         来了好几次了，味道还不错，两个人，吃的好饱，还剩不少
529    0  这是第二次团购这家的汤锅了，第一次去感觉还不错，第二次就继续回购了。位置依旧好找，对于熟悉南...
642    0                                味道挺好的，就是我们吃不了辣，剩了很多
715    0  嗯，去吃了以后感觉非常不错，分量也比较多，我们4个人去吃了，还没有吃完还剩了很多，比较经济便...
761    0  #椒盐皮皮虾#没肉，而且咸的发苦，巨咸无比！吃了一只剩下的全部都浪费了#香辣耗儿鱼#一个字“...
1335   0                               一如既往的好。味道好，6个人吃还剩了好多
1411   0  我们两个人团的这个四人餐，剩了四个菜基本没吃就已经吃到撑了。味道很赞，老板也很好，还送了一壶...
1660   0  该位置很好找，就在中渝爱都会的门口，我们九点过才去吃，没有排队，味道很好吃，我们两个人吃的中...
1675   0  好评！价格很实惠，分量很足，我们3个人吃的4人餐，还剩了很多，味道不错，老板很热情，还送了一...
1682   0                               味道不错，面块很好吃。四个人吃的，剩了些
1744   0  两个人团了三人餐，我们俩算是胃口很大的了，吃的还有剩，1只足够了。最开始看着量不大，但真的份...
1767   0                                       是人吃的吗？剩饭，剩菜，
1780   0  第一次别人送舍友的烤鸭。吃了就忘不了了，以后班里的聚会都从三峡带份烤鸭，5个人每次都能剩下。...
1907   0                                

In [16]:
key_word3=['骚味','苍蝇','虫','臭','想吐','太硬']
for i in key_word3:
  print (r0[r0['comment'].str.contains(i)])

      id                                            comment
1413   0  不好说的。豆花可以吃。烧菜只吃了一坨真的。建议老板自己尝一下。不晓得是撒子味道好重的骚味！第...
1985   0                                 有股羊骚味，我点的牛肉啊？送的汤好喝
      id                                        comment
1342   0             有苍蝇。。。。。。对于一个才学了节肢动物的人来说  真的是人都不好了
1753   0  去了两次，人太多，团购的上菜较慢，有苍蝇兄也有蚊子兄，总的来说算不错了！喜欢喝老鸭汤主要！
      id                                            comment
501    0  其实我们经常来，一般聚会请客都来这里吃，每个月都会来几次的。这次家里人吃就团了一个结果遇到下...
1963   0               完美，你家东西只有完美来形容，还附赠昆虫野味，完美到我再吃你家东西是SB
      id                                            comment
531    0  第一次没看评价点了这家，因为图片和价位还蛮吸引人的，吃完没事干给你们来个具体的建议，是真的不...
1296   0  这家冒菜确实难吃。 盐巴像打倒了一样 把我咸安逸了。这都算了嘛 关键是菜不晓得放了好多天的了...
      id                                            comment
613    0                                    不好吃，咖喱不正宗，想吐的味道
722    0                          想吐，银耳汤杯盖上面有一颗老鼠屎，而且还黏在上面了
1111   0  #精瘦肉300g/份#这个精瘦肉我真心不想吐槽，全是肉筋，平时我可以炒两份吃，结果这个精瘦肉...
1556   0                                  超级难吃 不想吐槽什

In [18]:
a=[17,1753,229,1767,238,1953 ,581, 697,722,1369, 
  1985,1342,753,1963,74,1296,531,1296,613,662 ,322,1413]

In [19]:
for i in a:
  svm.loc[i,('comment')]=[1]
svm.to_csv('后处理svm.csv', index=False)
print ('结束3.')

结束3.
